In [4]:
from pymongo import MongoClient
import polars as pl

from datetime import datetime
import pprint 
from bson.objectid import ObjectId

In [5]:
# Connect to the MongoDB server
client = MongoClient('mongodb://localhost:27017/')

db = client['event_database']
collection = db['events']

In [6]:
# Checkout databases and collections
print("Databases: ",client.list_database_names())
print("Collections: ",db.list_collection_names())

Databases:  ['admin', 'config', 'event_database', 'local', 'test_database']
Collections:  ['events']


In [ ]:
# Delete a database
client.drop_database('')

# 2. Query
- Define events of interest
- Select a date range
- query body
- Discover data type

In [7]:
# Define the events of interest
events_of_interest = ['Picking Up Dark Matter', 'Sleeping', 'Patting',
       'Claim Daily Rewards', 'Waking Up', 'Feeding', 'Visit Sanctuary',
       'Shop Transaction', 'Calling Pet', 'Swap Wearable',
       'Equip Wearable', 'Detach Wearable']

In [19]:
# Define the query pipeline
pipeline = [
    {
        '$match': {
            'event': {'$in': ['Claim Daily Rewards']}
        }
    },
    {
        '$group': {
            '_id': '$properties.distinct_id',
            'count': {'$sum': 1}
        }
    },
]


In [38]:
# Assuming you have already executed the pipeline and obtained the result
result = list(collection.aggregate(pipeline))

# Create the Polars DataFrame
df = pl.DataFrame(result)
df = df.sort("count", descending=True)
# Print the modified DataFrame
print(f"Unique Ids: {df.shape[0]}")

Unique Ids: 304


In [40]:
print(df.head(10))

shape: (10, 2)
┌───────────────────────────────────┬───────┐
│ _id                               ┆ count │
│ ---                               ┆ ---   │
│ str                               ┆ i64   │
╞═══════════════════════════════════╪═══════╡
│ c4ed373b-a956-4120-97b5-e70eb907… ┆ 2     │
│ 3d994231-d561-4a43-b999-7cca20f3… ┆ 2     │
│ 4864ca5d-c47b-4de6-95ed-7ce3e7ef… ┆ 2     │
│ 7c273de3-61e8-4eff-84ba-957e5533… ┆ 2     │
│ …                                 ┆ …     │
│ b83bc8cf-4f58-4bd6-88b7-54a605a9… ┆ 2     │
│ 9a6b6ee5-151f-437f-ab9a-52ee31b6… ┆ 2     │
│ 6f3845e6-8266-4ac0-8da6-a0313ebb… ┆ 2     │
│ a6b6411d-b3de-49f6-98f3-c12e2830… ┆ 2     │
└───────────────────────────────────┴───────┘
